<a href="https://colab.research.google.com/github/kolayn808/OpenAttack/blob/main/XAI_Train_RoBERTa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/MyDrive/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive


In [2]:
!pip install git+https://github.com/huggingface/transformers
!pip list | grep -E 'transformers|tokenizers'
!pip install textacy
!pip install transformers[torch]
!pip install accelerate -U

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-e202fl1z
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-e202fl1z
  Resolved https://github.com/huggingface/transformers to commit 58e7f9bb2faf30622c9bead7adf472ac59f3d301
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
tokenizers                       0.15.0
transformers                     4.36.0.dev0
ERROR: Operation cancelled by user


In [3]:
import sys
from gensim.corpora import WikiCorpus

def make_corpus(in_f, out_f):

    """Convert wikipedia xml dump file to text corpus"""

    output = open(out_f, 'w')
    print('starting corpus')
    wiki = WikiCorpus(in_f)
    print('loaded corpus')

    i = 0
    for text in wiki.get_texts():
        output.write(bytes(' '.join(text), 'utf-8').decode('utf-8') + '\n')
        i = i + 1
    if (i % 100 == 0):
        print('Processed ' + str(i) + ' articles')
    output.close()
    print('Processing complete!')

In [4]:
%%time
from pathlib import Path

from tokenizers import ByteLevelBPETokenizer

paths = ['wiki_en_10.txt']

# Initialize a tokenizer
tokenizer = ByteLevelBPETokenizer()

# Customize training
tokenizer.train(files=paths, vocab_size=52_000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

CPU times: user 1.52 s, sys: 140 ms, total: 1.66 s
Wall time: 1.98 s


# Новый раздел

In [5]:

!mkdir wikipedia_model_3e
tokenizer.save_model("wikipedia_model_3e")

mkdir: cannot create directory ‘wikipedia_model_3e’: File exists


['wikipedia_model_3e/vocab.json', 'wikipedia_model_3e/merges.txt']

In [6]:
%cd lm_training_experiments/

[Errno 2] No such file or directory: 'lm_training_experiments/'
/content/drive/MyDrive


In [7]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing


tokenizer = ByteLevelBPETokenizer(
    "./wikipedia_model_3e/vocab.json",
    "./wikipedia_model_3e/merges.txt",
)

tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=512)


tokenizer.encode("I am a fireman.").tokens

['<s>', 'I', 'Ġam', 'Ġa', 'Ġfire', 'man', '.', '</s>']

In [8]:
!nvidia-smi

Thu Dec  7 17:39:23 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [9]:
# Check that PyTorch sees it
import torch
torch.cuda.is_available()

True

In [10]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=52_000,
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)

In [11]:
from transformers import RobertaTokenizerFast

tokenizer = RobertaTokenizerFast.from_pretrained("./wikipedia_model_3e", max_len=512)

In [12]:
from transformers import RobertaForMaskedLM

model = RobertaForMaskedLM(config=config)

In [13]:
model.num_parameters()

83504416

In [14]:
%%time
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="./wiki_en_10.txt",
    block_size=128,
)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:119: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


CPU times: user 4.6 s, sys: 346 ms, total: 4.95 s
Wall time: 7.65 s


In [15]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [16]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./wikipedia_model_3e",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=64,
    save_steps=10_000
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset
)

In [17]:
%%time
trainer.train()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss


CPU times: user 3min 40s, sys: 607 ms, total: 3min 40s
Wall time: 3min 52s


TrainOutput(global_step=183, training_loss=8.31011129077015, metrics={'train_runtime': 232.0477, 'train_samples_per_second': 49.878, 'train_steps_per_second': 0.789, 'total_flos': 383756619497472.0, 'train_loss': 8.31011129077015, 'epoch': 3.0})

In [18]:
trainer.save_model("./wikipedia_model_3e")

In [19]:
from transformers import pipeline

unmasker = pipeline(
    "fill-mask",
    model="./wikipedia_model_3e",
    tokenizer="./wikipedia_model_3e"
)

In [20]:
#! git clone https://github.com/epfml/interpret-lm-knowledge.git
%cd /content/drive/MyDrive/interpret-lm-knowledge/scripts
from run_training_kg_experiments import *

run_experiments(tokenizer, model, unmasker, experiment_model, extractor, "Roberta3e")

/content/drive/MyDrive/interpret-lm-knowledge/scripts
